In [27]:
import pandas as pd
import numpy as np

import sys
sys.path.append('../')
from utils import *

pd.set_option('display.max_columns', None)

In [28]:
DATA_PATH = '../../data/'
PASSES_DF_PATH = DATA_PATH + 'passes_df.pkl'
WYSCOUT_PATH = DATA_PATH + 'wyscout/'
SKILLCORNER_PATH = DATA_PATH + 'skillcorner/'
MATCH_IDS_PATH = DATA_PATH + 'matchids.csv'
XT_PLOT_PATH = DATA_PATH + 'smoothed_xt.csv'

# Passes Network

In [29]:
passes_df = pd.read_pickle(PASSES_DF_PATH)

In [30]:
passes_df

,matchPeriod,player.id.wyscout,player.id.skillcorner,pass.recipient.id.wyscout,pass.recipient.id.skillcorner,location.x,location.y,pass.endLocation.x,pass.endLocation.y,play_direction,dxt,tracking.object_id,tracking.x,tracking.y,tracking.z,tracking.is_self,tracking.is_teammate,responsibility
frame,,,,,,,,,,,,,,,,,,
39,1,118,7387,625,7017,-1.058824,0.000000,8.470588,21.616842,TOP_TO_BOTTOM,0.003662,-1,0.40,0.29,0.2,False,False,0.0
39,1,118,7387,625,7017,-1.058824,0.000000,8.470588,21.616842,TOP_TO_BOTTOM,0.003662,7604,-41.03,-0.05,0.0,False,False,0.0
39,1,118,7387,625,7017,-1.058824,0.000000,8.470588,21.616842,TOP_TO_BOTTOM,0.003662,16320,-15.86,-7.61,0.0,False,False,0.0
39,1,118,7387,625,7017,-1.058824,0.000000,8.470588,21.616842,TOP_TO_BOTTOM,0.003662,11516,-16.08,2.42,0.0,False,False,0.0
39,1,118,7387,625,7017,-1.058824,0.000000,8.470588,21.616842,TOP_TO_BOTTOM,0.003662,11630,-14.54,-18.87,0.0,False,False,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57458,2,118,7387,221069,10916,-31.764706,-21.616842,-40.583333,-5.636923,BOTTOM_TO_TOP,0.064346,20191,31.49,-1.03,0.0,False,False,0.0
57458,2,118,7387,221069,10916,-31.764706,-21.616842,-40.583333,-5.636923,BOTTOM_TO_TOP,0.064346,12229,31.89,8.13,0.0,False,False,0.0
57458,2,118,7387,221069,10916,-31.764706,-21.616842,-40.583333,-5.636923,BOTTOM_TO_TOP,0.064346,12246,35.56,12.33,0.0,False,False,0.0


Keep only players of the opposing team, who are defenders with responsibility greater than 0.

In [17]:
passes_df = passes_df[~passes_df['tracking.is_teammate']]
passes_df = passes_df[passes_df['responsibility'] > 0]
# passes_df = passes_df[passes_df['tracking.player_position'].isin(defender_positions)]

In [18]:
# lineup_df = pd.read_csv(SKILLCORNER_PATH + str(skillcorner_id) + "_lineup.csv")
# lineup_df = lineup_df[['player_id', 'player_position']].rename(columns={'player_id': 'tracking.object_id', 'player_position': 'tracking.player_position'})
# lineup_df['tracking.player_position'].unique()
# passes_df_new = passes_df.merge(lineup_df, how='left')
# assert len(passes_df_new) == len(passes_df)
# passes_df = passes_df_new

# defender_positions = ['Right Wing Back', 'Right Center Back', 'Left Center Back', 'Left Wing Back']
# passes_df = passes_df[passes_df['tracking.player_position'].isin(defender_positions)]

# Passes Network

In [19]:
column_mask = ['player.id.skillcorner', 'pass.recipient.id.skillcorner', 'location.x', 'location.y' , 'pass.endLocation.x', 'pass.endLocation.y', 'dxt', 'responsibility']
group_column_mask = [col for col in column_mask if col != 'responsibility']
passes_network = passes_df[column_mask].groupby(group_column_mask).agg({'responsibility': 'sum'}).reset_index()

In [20]:
passes_network.sample(5)

,player.id.skillcorner,pass.recipient.id.skillcorner,location.x,location.y,pass.endLocation.x,pass.endLocation.y,dxt,responsibility
165,7863,10082,-30.705882,-20.888421,-48.833333,18.937778,-0.000768,3.954545
44,2548,12654,4.235294,10.382222,10.588235,25.987368,-0.003139,0.477603
570,31930,10082,-23.294118,27.444211,-29.647059,31.814737,-0.000567,0.242098
210,10738,2106,4.235294,-19.548889,-10.588235,-5.636923,0.010665,0.794072
416,12246,11630,7.411765,24.530526,16.941176,23.073684,0.008514,0.333493


# Defender Responsibility Network

In [8]:
tracking_object_id = 11606

player_passes_df = passes_df[passes_df['tracking.object_id'] == tracking_object_id]
defender_responsibility_network = player_passes_df[player_passes_df['responsibility'] > 0]

column_mask = ['player.id.skillcorner', 'pass.recipient.id.skillcorner', 'location.x', 'location.y' , 'pass.endLocation.x', 'pass.endLocation.y', 'dxt', 'responsibility']
defender_responsibility_network = defender_responsibility_network[column_mask]

In [9]:
defender_responsibility_network.sample(5)

,player.id.skillcorner,pass.recipient.id.skillcorner,location.x,location.y,pass.endLocation.x,pass.endLocation.y,dxt,responsibility
frame,,,,,,,,
11376,10082,7017,-9.529412,16.493333,4.235294,15.271111,0.005720,0.236144
24276,7863,10082,-30.705882,-20.888421,-48.833333,18.937778,-0.000768,0.365742
3093,10738,10082,20.117647,-14.048889,-19.058824,26.715789,-0.024542,0.171607
11367,7017,10082,4.235294,18.326667,-9.529412,16.493333,-0.005023,0.174165
24101,24466,10082,-5.294118,23.073684,-21.176471,31.086316,-0.003824,0.065480


# Aggregate Statistics for each Defender

In [10]:
defender_stats = passes_df.groupby('tracking.object_id').agg(
    average_responsibility=('responsibility', 'mean'),
    average_dxt=('dxt', 'mean'),
    group_size=('responsibility', 'size')
).reset_index()

defender_stats.sample(5)

,tracking.object_id,average_responsibility,average_dxt,group_size
7,6028,0.361673,0.000263,141
21,12229,0.335703,0.001701,14
22,12246,0.292880,-0.000492,9
1,946,0.304088,-0.000999,99
6,5922,0.394563,0.000992,95


# Defender Dyads

In [11]:
passes_df_cp = passes_df.reset_index()
pass_filt_df = passes_df_cp.groupby('frame').filter(lambda x: len(x) >= 2)

relevant_cols = ['frame', 'tracking.object_id_x', 'tracking.object_id_y',  'dxt_x', 'responsibility_x', 'responsibility_y']
joint_df = pass_filt_df.merge(pass_filt_df, on=['frame'])[relevant_cols]
joint_df = joint_df[joint_df['tracking.object_id_x'] < joint_df['tracking.object_id_y']]
joint_df['joint_resp'] = joint_df['responsibility_x'] + joint_df['responsibility_y']
joint_df.rename(columns={'dxt_x': 'dxt'}, inplace=True)

In [12]:
columns_joint_group = ['frame', 'tracking.player.id.skillcorner_x', 'tracking.player.id.skillcorner_y', 'joint_resp']

defender_dyads_network = joint_df.groupby(['tracking.object_id_x', 'tracking.object_id_y']).agg(
    joint_responsibility_mean=('joint_resp', 'mean'),
    joint_responsibility_sum=('joint_resp', 'sum'),
    group_size=('joint_resp', 'size')
    ).reset_index()

In [13]:
defender_dyads_network.sample(5)

,tracking.object_id_x,tracking.object_id_y,joint_responsibility_mean,joint_responsibility_sum,group_size
96,7863,10082,0.840880,0.840880,1
49,3837,14397,0.843588,0.843588,1
166,20191,22103,0.832722,3.330888,4
6,277,11606,0.763308,31.295641,41
4,277,8206,0.629294,1.258588,2
